# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

## Importaciones

In [260]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [261]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

## 1. Solución aleatoria

### Funciones globales auxiliares

In [262]:
def get_contrapart(index):
    """ Obtiene qué indice de la celda es la contraparte del índice actual. """
    if index == 0:
        return 2
    elif index == 1:
        return 1
    elif index == 2:
        return 0
    elif index == 3:
        return 3
    
def random_number(min, max):
    """ Regresa un número aleatorio dentro del límite. """
    return np.random.randint(min, max)

### Implementación del agente

In [263]:
class FirefighterAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.carrying_victim = False
        self.knocked_down = False
        self.points_saved = 0

    
    def step(self):
        print(f"El agente {self.unique_id} ha comenzado su turno.")
        
        if self.action_points > 0:
            # Asignar un número a cada función y que elija aleatoriamente
            option = random_number(0, 7)
            self.move_to_cell()
            self.toggle_door(self.pos)
            self.extinguish(self.pos, 1)
            self.extinguish(self.pos, 2)
            self.extinguish(self.pos, 3)
            self.chop(self.pos, random_number(0, 3))
            self.toggle_victim()
            self.save_points()

         # Fin de turno
        elif self.action_points == 0:
            self.end_turn()
        


    def move_to_cell(self):
        """ Mueve al agente a una celda adyacente que no esté bloqueada por una puerta o pared. """
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        print(f"Posibles posiciones: {possible_positions} for agent {self.unique_id}")
        indexes = [x for x in range( len(possible_positions) )]
        np.random.shuffle(indexes)

        for i in indexes:
            position = possible_positions[i]
            status = self._get_cell_status(position)
            walls_doors = self._check_walls_doors(position, i)

            if not walls_doors and self.model.grid.is_cell_empty(position):
                if status == 2 and not self.carrying_victim:
                    self.model.grid.move_agent(self, position)
                    self.action_points -= 2
                elif self.carrying_victim and status != 2:
                    self.action_points -= 2
                    self.model.grid.move_agent(self, position)
                else:
                    self.action_points -= 1
                    self.model.grid.move_agent(self, position)

                self._check_for_poi(self.pos)

                break

  
    def toggle_door(self,position):
        """ Cambia el estado de la puerta en la celda (x, y). """
        x, y = position
        cell = self.model.doors[x][y]

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Encontrar la puerta que se quiere alterar
        # 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        # 1 Abierto 2 Cerrado
        if self.action_points >= 1:
            index = 0
            for i in cell:
                if i != 0:
                    if i == 1:
                        cell[index] = 2
                        new_state = 2
                    else:
                        cell[index] = 1
                        new_state = 1

                    self.action_points -= 1
                    break
                index += 1
            
            # Encontrar la otra celda de la puerta
            if index == 0:
                other_cell = possible_positions[0]
                other_index = 2
            elif index == 2:
                other_cell = possible_positions[2]
                other_index = 0
            elif index == 1: # izquierda
                other_cell = possible_positions[3]
                other_index = 3
            elif index == 3:
                other_cell = possible_positions[1]
                other_index = 1
            else:
                return
            
            # Cambiar el estado de la otra celda
            next_cell = self.model.doors[other_cell[0]][other_cell[1]]
            next_cell[other_index] = new_state
            return
    
    
    def extinguish(self, position, option):
        """ Extingue el fuego o el humo en la celda (x, y). """
        x, y = position
        cell = self.model.grid_status[x][y]
        
        if option == 1 and cell == 2 and self.action_points >= 2:
            self.model.grid_status[x][y] = 0
            self.action_points -= 2
        elif option == 2 and cell == 2 and self.action_points >= 1:
            self.model.grid_status[x][y] = 1
            self.action_points -= 1
        elif option == 3 and cell == 1 and self.action_points >= 1:
            self.model.grid_status[x][y] = 0
            self.action_points -= 1
        else:
            return
            

    def chop(self, position, index):
        """ Daña la pared en la celda (x, y). """
        x, y = position
        xs, ys = self.pos
        other_index = get_contrapart(index)

        wall = self.model.walls[x][y][other_index]
        
        if wall > 0 and self.action_points >= 2:
            self.model.walls[x][y][other_index] -= 0.5
            self.model.walls[xs][ys][index] -= 0.5
            self.action_points -= 2
            self.model.damage_counters -= 1


    def toggle_victim(self):
        """ Toma o deja a la víctima en la celda (x, y). """
        x, y = self.pos
        cell = self.model.poi[x][y]

        if cell == 2 and not self.carrying_victim:
            self.carrying_victim = True
            self.model.poi[x][y] = 0

        elif cell == 0 and self.carrying_victim:
            self.carrying_victim = False
            self.model.poi[x][y] = 2


    def save_points(self):
        self.points_saved = self.action_points
        self.end_turn()


    def end_turn(self):
        """ Finaliza turno del agente, pasando a la siguiente fase de juego y al siguiente jugador. """
        print(f"El agente {self.unique_id} ha terminado su turno.")
        self.action_points = 4 + self.points_saved
        self.points_saved = 0
        if self.action_points > 8:
            self.action_points = 8

        self.model.game_phase = "Fire Advance"

        if self.model.active_player < 5:
            self.model.active_player += 1
        else:
            self.model.active_player = 0


    # Métodos auxiliares
    def _check_for_poi(self, position):
        """ Verifica si la celda (x, y) tiene un punto de interés y qué tipo de punto es. """
        x, y = position
        cell = self.model.poi[x][y]
        choice = self.model._choose_between_victim_or_false_alarm()

        if cell == 1:
            self.model.poi[x][y] = choice
            self.model.point_of_interest -= 1



    def _check_walls_doors(self, position, index):
        """ Verifica si la celda (x, y) tiene una pared o puerta que impida el paso. """
        x, y = position
        other_index = get_contrapart(index)
        
        cell_doors = self.model.doors[x][y]
        cell_walls = self.model.walls[x][y]
        
        door = cell_doors[other_index]
        wall = cell_walls[other_index]
            
        if door == 2 or wall == 1 or wall == 0.5:
            return True
        else:
            return False
        
        
    def _get_cell_status(self, position):
        """ Obtiene el contenido de una celda (humo o fuego). """
        x, y = position
        return self.model.grid_status[x][y]
    

### Implementación de modelo

In [ ]:
class FlashPointModel(Model):

    def __init__(self, walls_matrix, doors_matrix, outside_matrix, 
                 width = 10, height = 8, agents = 6, damage_counters = 24):

        super().__init__()
        self.grid = SingleGrid(height, width, torus = False)
        self.schedule = SimultaneousActivation(self)

        self.datacollector = DataCollector()

        # Matrices
        self.walls = walls_matrix
        self.doors = doors_matrix
        self.outside_doors = outside_matrix
        self.grid_status = np.zeros((8, 10))
        self.poi = np.zeros((8, 10))

        self.saved_victims = 0
        self.lost_victims = 0
        self.damage_counters = damage_counters
        self.false_alarms = 6
        self.total_victims = 12
        self.point_of_interest = 3
        self.steps = 0
        self.width = width
        self.height = height
        self.game_phase = "Player"
        self.agents_by_id = {}
        self.active_player = 0
        self.victory = False
        self.defeat = False

        self.edge_positions = self.get_edge_positions(width, height)

        print(self.grid)
        # Creación agentes
        for i in range(agents):
            agent = FirefighterAgent(i, self)
            self.schedule.add(agent)
            self.agents_by_id[i] = agent

            # Posicionar
            agent_placed = False
            while not agent_placed:
                chosen_pos = random.choice(self.edge_positions)
                print(f"Posición elegida: {chosen_pos}")
                if self.grid.is_cell_empty(chosen_pos):
                    self.grid.move_agent(agent, chosen_pos)
                    agent_placed = True
                    print(f"El agente {i} ha sido colocado en la posición {agent.pos}.")

        
        # Posicionar fuegos
        fire_count = 10
        while fire_count > 0:
            x, y = self.roll_dice()
            cell = self.grid_status[x][y]
            if cell == 0:
                self.grid_status[x][y] = 2
                fire_count -= 1
                

        # Posicionar POIs
        POI_count = 3
        while POI_count > 0:
            x, y = self.roll_dice()
            cell_poi = self.poi[x][y]
            cell_status = self.grid_status[x][y]
            if cell == 0 and cell_status != 2:
                self.poi[x][y] = 1
                POI_count -= 1


    def get_edge_positions(self, width, height):
        """ Obtiene las celdas de los bordes del tablero. """
        edge_positions = []
        for x in range(0, height - 1):
            edge_positions.append((x, 0))  # Orilla superior
            edge_positions.append((x, 9))  # Orilla inferior

        for y in range(0, width - 1):
            edge_positions.append((0, y))  # Orilla izquierda
            edge_positions.append((7, y))  # Orilla derecha

        return edge_positions
    

    def advance_fire(self):
        """ Coloca un humo aletorio. """
        x, y = self.roll_dice()
        cell = self.grid_status[x][y]
        
        # Si la celda está vacía, se coloca humo
        if cell == 0:
            self.grid_status[x][y] = 1
        elif cell == 1:
            self.grid_status[x][y] = 2
        elif cell == 2:
            self.explosion(x, y)
        
        self.game_phase = "Secondary Effects"


    def explosion(self, x, y):
        """ Realiza una explosión y llama a explosion_effect para las celdas adyacentes. """
        adjacent_cells = self.model.grid.get_neighborhood((x, y), moore = False, include_center = False)
        
        index = 0
        for position in adjacent_cells:
            self.explosion_effect(position[0], position[1], index, x, y)
            index += 1
        
        
    def explosion_effect(self, adj_x, adj_y, index, current_x, current_y):
        """ Verifica qué encuentra en las celdas adyacentes y genera el efecto correspondiente. """
        cell = self.grid_status[adj_x][adj_y]
        other_index = get_contrapart(index)
        door = self.doors[adj_x][adj_y][other_index]
        wall = self.walls[adj_x][adj_y][other_index]
        
        # Puerta cerrada se destruye
        if door == 2:
            self.doors[adj_x][adj_y][other_index] = 0
            self.doors[current_x][current_y][index] = 0  

        # Puerta abierta se destruye y avanza fuego
        elif door == 1:
            self.doors[adj_x][adj_y][other_index] = 0
            self.doors[current_x][current_y][index] = 0
            self.grid_status[adj_x][adj_y] = 2

        # Pared se daña
        elif wall == 1 or wall == 0.5:
            self.walls[adj_x][y][other_index] -= 0.5
            self.walls[current_x][current_y][index] -= 0.5
            self.damage_counters -= 1

        elif wall == 0:
            if cell == 2:
                self.shockwave(adj_x, adj_y, index)
            else:
                self.grid_status[adj_x][adj_y] = 2
                
                
    def shockwave(self, initial_x, initial_y, index):
        """ Realiza una onda expansiva de llammas en una dirección hasta no encontrar más fuego."""
        keep_going = True
        current_x, current_y = initial_x, initial_y

        while keep_going:
            adjacent_cells = self.model.grid.get_neighborhood((current_x, current_y), moore = False, include_center = False)
            # La siguiente celda es siempre en la misma dirección
            next_fire = adjacent_cells[index]
            next_x, next_y = next_fire

            # Shockwave avanza a través del fuego
            if self.grid_status[next_x][next_y] == 2:
                current_x, current_y = next_fire
            else:
                keep_going = False  
                other_index = get_contrapart(index)
                door = self.doors[current_x][current_y][index]
                wall = self.walls[current_x][current_y][index]
                
                # Puerta cerrada se destruye
                if door == 2:
                    self.doors[next_x][next_y][other_index] = 0
                    self.doors[current_x][current_y][index] = 0  

                # Puerta abierta se destruye y avanza fuego
                elif door == 1:
                    self.doors[next_x][next_y][other_index] = 0
                    self.doors[current_x][current_y][index] = 0
                    self.grid_status[next_x][next_y] = 2

                # Pared se daña
                elif wall == 1 or wall == 0.5:
                    self.walls[next_x][next_y][other_index] -= 0.5
                    self.walls[current_x][current_y][index] -= 0.5
                    self.damage_counters -= 1

                # Se coloca fuego continuo
                else:
                    self.grid_status[next_x][next_y] = 2


    def secondary_effects(self):
        """ Verifica los efectos secundarios tras la aparición de humo o fuego. """
        self.flashover()
        self.knock_down()
        self.check_poi_on_fire()
        self.game_phase = "Replenish POI"


    def knock_down(self):
        """ Verifica si un bombero es derribado por el fuego y lo coloca en la puerta más cercana. """
        for agent in self.schedule.agents:
            x, y = agent.pos
            cell = self.grid_status[x][y]
            if cell == 2:
                if agent.carrying_victim:
                    self.lost_victims += 1
                    agent.carrying_victim = False
                agent.knocked_down = True

                # Colocarlo en la puerta externa más cercana
                self.grid.move_agent(agent, self.get_closest_door(x, y))


    def get_closest_door(self, x, y):
        """ Obtiene la puerta externa más cercana a la celda (x, y). """
        min_distance = float('inf')
        closest = None

        for door in self.outside_doors:
            distance = abs(x - door[0]) + abs(y - door[1])
            if distance < min_distance:
                min_distance = distance
                closest = door

        if closest[0] == 1:
            return (closest[0] - 1, closest[1])
        elif closest[0] == 6:
            return (closest[0] + 1, closest[1])
        elif closest[1] == 1:
            return (closest[0], closest[1] - 1)
        elif closest[1] == 8:
            return (closest[0], closest[1] + 1)


    def flashover(self):
        """ Convertir cualquier humo pegado a fuego en fuego."""
        for x in range(self.width):
            for y in range(self.height):

                if self.grid_status[x][y] == 1:
                    adjacent_cells = self.model.grid.get_neighborhood((x, y), moore = False, include_center = False)

                    for position in adjacent_cells:
                        if self.grid_status[position[0]][position[1]] == 2:
                            self.grid_status[x][y] = 2
                            break


    def check_poi_on_fire(self):
        """ Verifica si un punto de interés es afectado por el fuego. """
        for x in range(self.width):
            for y in range(self.height):

                if self.poi[x][y] != 0 and self.grid_status[x][y] == 2:
                    if self.poi[x][y] == 1:
                        choice = self._choose_between_victim_or_false_alarm()
                        if choice == 2:
                            self.lost_victims += 1

                    elif self.poi[x][y] == 2:
                        self.lost_victims += 1

                    self.poi[x][y] = 0
                    self.point_of_interest -= 1

        


    def replenish_poi(self):
        """ Reponer los puntos de interés en las celdas vacías. """
        while self.point_of_interest < 3 and (self.total_victims > 0 or self.false_alarms > 0):
            poi_placed = False

            while not poi_placed:
                x, y = self.roll_dice()
                cell_poi = self.poi[x][y]
                cell_status = self.grid_status[x][y]

                if cell_poi == 0:
                    if cell_status != 0:
                        self.grid_status[x][y] = 0
                        self.poi[x][y] = 1

                    elif self.grid.is_cell_empty((x, y)):
                        self.poi[x][y] = self._choose_between_victim_or_false_alarm()
                        
                    self.point_of_interest += 1
                    poi_placed = True
        
        self.game_phase = "Player"


    def _choose_between_victim_or_false_alarm(self):
        " Elige si el punto de interés es una víctima o una falsa alarma. "
        while self.total_victims > 0 or self.false_alarms > 0:
            choice = np.random.randint(2, 3)
            if choice == 2 and self.total_victims > 0:
                self.total_victims -= 1
                return choice
            elif choice == 3 and self.false_alarms > 0:
                self.false_alarms -= 1
                return choice
        
        
    def roll_dice(self):
        """ Regresa un número aleatorio entre 1 y 8 y del 1 al 6. """
        return self.random.randint(1, 6), self.random.randint(1, 8)
    

    def is_game_over(self):
        """ Verifica si el juego ha terminado por victoria o derrota. """
        if self.damage_counters <= 0 or self.lost_victims >= 4:
            self.defeat = True
            return True
        elif self.saved_victims >= 7:
            self.victory = True
            return True
        else:
            return False
    

    def step(self):
        print("Dentro de step modelo ", self.steps)
        if self.game_phase == "Player":
            agent = self.agents_by_id[self.active_player]
            agent.step()
        elif self.game_phase == "Advance Fire":
            self.advance_fire()
        elif self.game_phase == "Secondary Effects":
            self.secondary_effects()
        elif self.game_phase == "Replenish POI":
            self.replenish_poi()

        self.datacollector.collect(self)
        self.steps += 1


In [265]:
walls_matriz = [
    [[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [0, 1, 0, 1], [1, 0, 0, 0], [1, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 0, 1], [1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[0, 1, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [1, 0, 0, 1], [1, 1, 0, 1], [1, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[0, 1, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[1, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [1, 1, 0, 1], [1, 1, 0, 0], [1, 0, 0, 1], [1, 1, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]],
    [[0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
]

doors_matriz = np.zeros((8, 10, 4))
outside_matriz = [(1, 1), (1, 8), (6, 1), (6, 8)]

### Simulación aleatoria

In [ ]:
model = FlashPointModel(walls_matriz, doors_matriz, outside_matriz)

while not model.is_game_over():
    model.step()

print("Steps: ", model.steps)

Posición elegida: (0, 5)
El agente 0 ha sido colocado en la posición (0, 5).
Posición elegida: (6, 0)
El agente 1 ha sido colocado en la posición (6, 0).
Posición elegida: (1, 9)
El agente 2 ha sido colocado en la posición (1, 9).
Posición elegida: (6, 0)
Posición elegida: (3, 9)
El agente 3 ha sido colocado en la posición (3, 9).
Posición elegida: (0, 8)
El agente 4 ha sido colocado en la posición (0, 8).
Posición elegida: (2, 0)
El agente 5 ha sido colocado en la posición (2, 0).
Dentro de step modelo  0
El agente 0 ha comenzado su turno.
Posibles posiciones: ((0, 4), (0, 6), (1, 5)) for agent 0
El agente 0 ha terminado su turno.
Dentro de step modelo  1
Dentro de step modelo  2
Dentro de step modelo  3
Dentro de step modelo  4
Dentro de step modelo  5
Dentro de step modelo  6
Dentro de step modelo  7
Dentro de step modelo  8
Dentro de step modelo  9
Dentro de step modelo  10
Dentro de step modelo  11
Dentro de step modelo  12
Dentro de step modelo  13
Dentro de step modelo  14
Dentr